<a href="https://colab.research.google.com/github/Shashikala-s/NLP/blob/main/XLM_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers dataset scikit-learn

In [ ]:
!pip install fsspec==2023.6.0
# !pip install --upgrade fsspec


In [ ]:
from transformers import XLMRobertaTokenizer , XLMRobertaForSequenceClassification, Trainer , TrainingArguments

from datasets import load_dataset

from sklearn.metrics import accuracy_score, f1_score

import  numpy  as np



In [ ]:
full_dataset2 =load_dataset("amazon_polarity", split="train")

full_dataset = full_dataset2.shuffle(seed=42).select(range(500))

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
dataset = full_dataset.train_test_split(test_size=0.5)

In [ ]:
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

In [ ]:
def preprocess(examples):
  return tokenizer(examples["content"], truncation=True)

In [ ]:
dataset =dataset.map(preprocess)

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
train_dataset = dataset["train"]
train_test = train_dataset.train_test_split(test_size=0.2)  # Correct: use train_dataset here
train_dataset = train_test["train"]
eval_dataset = train_test["test"]


In [ ]:
dataset =dataset.rename_column("label", "labels")

In [ ]:
dataset.set_format(type="torch")

In [ ]:
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_matrix(pred):
  lables =pred.label_ids
  preds =np.argmax(pred.predictions, axis=1)
  return {
    "accuracy:" : accuracy_score(lables, preds),
    "f1:" : f1_score(lables, preds, average="weighted")
  }

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    # evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=2e-5
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_matrix
)

In [ ]:
# 10. Train the model
trainer.train()

# 11. Evaluate
trainer.evaluate()

# 12. Inference (example in English or any language!)
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

print(classifier("I love this movie!"))  # English
print(classifier("J'adore ce film !"))   # French
print(classifier("Ich liebe diesen Film!"))  # German

ValueError: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.